In [1]:
import subprocess

# Define the PowerShell command
command = "Get-ChildItem -Recurse | Select-Object Name, LastWriteTime | Sort-Object LastWriteTime | Format-Table -AutoSize"

# Use subprocess to run the PowerShell command and redirect the output to history.txt
try:
    with open("history.txt", "w") as output_file:
        subprocess.run(["powershell", "-Command", command], stdout=output_file, stderr=subprocess.PIPE, text=True)
    print("Command executed successfully. Check 'history.txt' for results.")
except Exception as e:
    print(f"An error occurred: {e}")


Command executed successfully. Check 'history.txt' for results.


In [2]:
import re
from datetime import datetime, timedelta
from collections import defaultdict

# Specify idle time threshold as a timedelta (hours, minutes, seconds)
IDLE_TIME_THRESHOLD = timedelta(minutes=80)  # Lower threshold for more sensitivity

def parse_history_file(file_path):
    daily_times = defaultdict(list)

    # Define a regex pattern to capture the filename and timestamp
    pattern = re.compile(r'^(.*?)(\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2}:\d{2} (?:AM|PM))$')

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or 'LastWriteTime' in line:
                continue
            
            # Use regex to find the filename and timestamp
            match = pattern.match(line)
            if not match:
                continue
            
            filename, timestamp_str = match.groups()
            # Parse the timestamp
            try:
                timestamp = datetime.strptime(timestamp_str, '%m/%d/%Y %I:%M:%S %p')
                # Store the timestamp in the corresponding day's list
                daily_times[timestamp.date()].append(timestamp)
            except ValueError as e:
                print(f"Skipping invalid timestamp: {timestamp_str} - Error: {e}")

    return daily_times

def calculate_time_worked(daily_times, idle_time_threshold):
    time_worked_per_day = {}
    
    for date, timestamps in daily_times.items():
        timestamps.sort()  # Sort timestamps for accurate time calculation
        total_time = timedelta()
        last_timestamp = None
        chunks = []  # List to store valid time chunks

        for current_timestamp in timestamps:
            if last_timestamp is None:
                # Start the first chunk
                chunks.append([current_timestamp, None])  # [start, stop]
            else:
                # Calculate time difference
                time_diff = current_timestamp - last_timestamp
                
                if time_diff > idle_time_threshold:
                    # Time difference exceeds idle time, close current chunk
                    if chunks[-1][1] is None:  # Close the last chunk
                        chunks[-1][1] = last_timestamp
                    # Start a new chunk
                    chunks.append([current_timestamp, None])
            
            # Update last_timestamp for the next iteration
            last_timestamp = current_timestamp
        
        # Close the last chunk if it was not closed
        if chunks and chunks[-1][1] is None:
            chunks[-1][1] = last_timestamp  # Update end time of the last chunk

        # Calculate total time from the valid chunks
        total_time = sum((stop - start for start, stop in chunks if stop is not None), timedelta())
        
        time_worked_per_day[date] = (total_time, chunks)  # Store both total time and chunks

    return time_worked_per_day

def get_start_stop_times(daily_times):
    start_stop_times = {}
    
    for date, timestamps in daily_times.items():
        start_time = min(timestamps).time()
        stop_time = max(timestamps).time()
        start_stop_times[date] = (start_time, stop_time)

    return start_stop_times

if __name__ == '__main__':
    history_file_path = 'history.txt'  # Replace with your file path if necessary
    daily_times = parse_history_file(history_file_path)
    start_stop_times = get_start_stop_times(daily_times)
    time_worked_per_day = calculate_time_worked(daily_times, IDLE_TIME_THRESHOLD)

    for date, (start, stop) in start_stop_times.items():
        total_time_worked, time_chunks = time_worked_per_day[date]
        print(f"{date}: Start Time = {start}, Stop Time = {stop}, Total Time Worked = {total_time_worked}")

        # Print all valid start-stop chunks for the day
        for chunk_start, chunk_stop in time_chunks:
            print(f"  Chunk: Start = {chunk_start.time()}, Stop = {chunk_stop.time()}, Duration = {chunk_stop - chunk_start}")


2024-05-30: Start Time = 11:22:06, Stop Time = 11:24:14, Total Time Worked = 0:02:08
  Chunk: Start = 11:22:06, Stop = 11:24:14, Duration = 0:02:08
2024-09-27: Start Time = 23:13:08, Stop Time = 23:44:34, Total Time Worked = 0:31:26
  Chunk: Start = 23:13:08, Stop = 23:44:34, Duration = 0:31:26
2024-09-29: Start Time = 15:32:27, Stop Time = 15:32:27, Total Time Worked = 0:00:00
  Chunk: Start = 15:32:27, Stop = 15:32:27, Duration = 0:00:00
2024-09-30: Start Time = 16:13:53, Stop Time = 16:14:00, Total Time Worked = 0:00:07
  Chunk: Start = 16:13:53, Stop = 16:14:00, Duration = 0:00:07
2024-10-01: Start Time = 14:43:51, Stop Time = 14:46:48, Total Time Worked = 0:02:57
  Chunk: Start = 14:43:51, Stop = 14:46:48, Duration = 0:02:57
2024-10-02: Start Time = 16:08:08, Stop Time = 21:55:24, Total Time Worked = 0:20:59
  Chunk: Start = 16:08:08, Stop = 16:08:08, Duration = 0:00:00
  Chunk: Start = 18:55:46, Stop = 18:55:46, Duration = 0:00:00
  Chunk: Start = 21:34:25, Stop = 21:55:24, Durat

In [24]:
import re
from datetime import datetime, timedelta
from collections import defaultdict

# Specify idle time threshold in minutes
IDLE_TIME_MINUTES = 10

def parse_history_file(file_path):
    daily_times = defaultdict(list)

    # Define a regex pattern to capture the filename and timestamp
    pattern = re.compile(r'^(.*?)(\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2}:\d{2} (?:AM|PM))$')

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or 'LastWriteTime' in line:
                continue
            
            # Use regex to find the filename and timestamp
            match = pattern.match(line)
            if not match:
                continue
            
            filename, timestamp_str = match.groups()
            # Parse the timestamp
            try:
                timestamp = datetime.strptime(timestamp_str, '%m/%d/%Y %I:%M:%S %p')
                # Store the timestamp in the corresponding day's list
                daily_times[timestamp.date()].append(timestamp)
            except ValueError as e:
                print(f"Skipping invalid timestamp: {timestamp_str} - Error: {e}")

    return daily_times

def calculate_time_worked(daily_times, idle_time_minutes):
    time_worked_per_day = {}
    idle_time_duration = timedelta(minutes=idle_time_minutes)

    for date, timestamps in daily_times.items():
        timestamps.sort()  # Sort timestamps for accurate time calculation
        total_time = timedelta()
        last_timestamp = timestamps[0]

        for current_timestamp in timestamps[1:]:
            # Calculate time difference
            time_diff = current_timestamp - last_timestamp
            
            # Check if the time difference exceeds idle time
            if time_diff > idle_time_duration:
                # If it exceeds idle time, reset last_timestamp to current
                last_timestamp = current_timestamp
            else:
                # If within idle time, add to total time worked
                total_time += time_diff

            # Update last_timestamp for the next iteration
            last_timestamp = current_timestamp
        
        # Account for the last session
        total_time += timestamps[-1] - timestamps[0] if len(timestamps) > 1 else timedelta()
        
        time_worked_per_day[date] = total_time

    return time_worked_per_day

def get_start_stop_times(daily_times):
    start_stop_times = {}
    
    for date, timestamps in daily_times.items():
        start_time = min(timestamps).time()
        stop_time = max(timestamps).time()
        start_stop_times[date] = (start_time, stop_time)

    return start_stop_times

if __name__ == '__main__':
    history_file_path = 'history.txt'  # Replace with your file path if necessary
    daily_times = parse_history_file(history_file_path)
    start_stop_times = get_start_stop_times(daily_times)
    time_worked_per_day = calculate_time_worked(daily_times, IDLE_TIME_MINUTES)

    for date, (start, stop) in start_stop_times.items():
        print(f"{date}: Start Time = {start}, Stop Time = {stop}, Time Worked = {time_worked_per_day[date]}")


2024-10-06: Start Time = 23:41:06, Stop Time = 23:58:44, Time Worked = 0:22:04
2024-10-07: Start Time = 00:45:12, Stop Time = 19:27:29, Time Worked = 19:00:46
2024-10-08: Start Time = 09:04:29, Stop Time = 23:34:55, Time Worked = 15:20:50
2024-10-09: Start Time = 00:27:26, Stop Time = 00:45:09, Time Worked = 0:19:00
2024-10-10: Start Time = 10:58:51, Stop Time = 18:08:11, Time Worked = 7:37:54
